# es index 优化

## close index

### 测试


* 查看当前各个node segments使用情况

```
curl '172.21.2.40:9200/_cat/nodes?v&h=name,segments.count,segments.memory,segments.index_writer_memory,segments.version_map_memory,segments.fixed_bitset_memory'
name segments.count segments.memory segments.index_writer_memory segments.version_map_memory segments.fixed_bitset_memory
node138 0 0b 0b 0b 0b
node160 13644 304.6mb 0b 0b 0b
node161 13576 299.7mb 0b 0b 0b
node139 0 0b 0b 0b 0b
node159 13711 299.5mb 0b 0b 0b
```

* 执行索引关闭操作

``` 
curl  '172.21.2.40:9200/_cat/indices' -s |grep 2017 |wc -l

100

curl -XPOST '172.21.2.40:9200/*2017*/_close?pretty' -d ''

```


* 查看关闭索引后各个node segments使用情况

```
name segments.count segments.memory segments.index_writer_memory segments.version_map_memory segments.fixed_bitset_memory
node138 0 0b 0b 0b 0b
node160 11517 266.5mb 0b 0b 0b
node161 11534 263.7mb 0b 0b 0b
node139 0 0b 0b 0b 0b
node159 11644 263.1mb 3.9mb 0b 0b
```

* 检查各个node节点io、cpu和内存情况

未发现明显变化 （注意：如果打开索引时，如果数据量较大，cpu和io将有明显升高）




### 生产环境规划

* 先关闭2017年所有index

``` 
curl -XPOST '172.21.0.40:9200/*2017*/_cluse' -d ''
```

* 编写脚本实现每天定时关闭前120天的index

```
```


## force merge index

### 测试

* 


In [ ]:
!bash -x es_index_daily.sh

+ es_index_log=/tmp/es_index.log
++ date -d '1  days ago' +%Y.%m.%d
usage: date [-jnRu] [-d dst] [-r seconds] [-t west] [-v[+|-]val[ymwdHMS]] ... 
            [-f fmt date | [[[mm]dd]HH]MM[[cc]yy][.ss]] [+format]
+ yesterday=
++ date -d '120  days ago' +%Y.%m.%d
usage: date [-jnRu] [-d dst] [-r seconds] [-t west] [-v[+|-]val[ymwdHMS]] ... 
            [-f fmt date | [[[mm]dd]HH]MM[[cc]yy][.ss]] [+format]
+ daysago=
+ es_url=http://172.21.2.40:9200/
+ index_keywords=("application" "nginx_frontend" "nginx_backend")
++ date +%Y-%m-%d
+ echo 2018-04-17
+ curl -s 'http://172.21.2.40:9200/_cat/nodes?v&h=name,segments.count,segments.memory,segments.index_writer_memory,segments.version_map_memory,segments.fixed_bitset_memory'
